In [8]:
# Autoreload external files on execution
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Webscraping mit Pyhon: https://realpython.com/beautiful-soup-web-scraper-python/#scrape-the-fake-python-job-site

python packages:

```
requests
dataclasses
beautifulsoup4
```

In [9]:
import requests
import re
from bs4 import BeautifulSoup
from databaseHelper import Person

persons = []

UHH

In [10]:
uhh_url = "https://www2.informatik.uni-hamburg.de/fiona/pers.php"

page = requests.get(uhh_url)
soup = BeautifulSoup(page.content, "html.parser")

result = []
# get all rows of the person table (<tr>)
for person in soup.find_all(name="tr"):
    # skip all rows without an id attribute
    if not person.has_attr("id"):
        continue

    # all table field values are insde <a> tags
    fields = person.findAll("a")

    name = fields[1].text
    department = fields[2].text

    result.append(Person(
        name=name,
        university="Universität Hamburg",
        department=department
    ))

print(f"done: scraped {len(result)} UHH staff")
persons.extend(result)

done: scraped 325 UHH staff


HAW

In [11]:
haw_url = "https://www.haw-hamburg.de/hochschule/technik-und-informatik/departments/informatik/unser-department/beschaeftigte/"

page = requests.get(haw_url)
soup = BeautifulSoup(page.content, "html.parser")

result = []

# loop over table rows
for row in soup.findAll("div", class_="row person-tile filter_tile border-top pt-4"):
    # find cell containing the name
    person = row.find("div", class_="col-12")

    name = ""
    # extract title and name from html
    for tag in person.findAll("a", {"title" : "Zur Profilseite"}):
        # title and name are in seperate <b> tags
        for part in tag.findAll("b"):
            # remove whitespace
            name += " " + re.sub(r"\s+", " ", part.text.strip())

        result.append(Person(
            name=name.strip(),
            university="Hochschule für Angewandte Wissenschaften Hamburg"
        ))

print(f"done: scraped {len(result)} HAW staff")
persons.extend(result)

done: scraped 67 HAW staff


split name into title and name

for instance: 
`"Prof. Dr. Klaus-Peter Kossakowski"` becomes `"Prof. Dr.", "Klaus-Peter Kossakowski"`

In [12]:
for person in persons:
    # chatGPT kann super so dumme regexe schreiben 
    pattern = r"^(Prof\. Dr\.|Prof\.|Dr\.)(-Ing\.| rer\. nat\.)?(?=.)(?:\s|$)"
    
    title = re.search(pattern, person.name)
    
    if not title: continue

    person.title = title.group().strip()
    person.name = re.sub(pattern, "", person.name).strip()

insert persons into database

In [13]:
from databaseHelper import PersonDatabaseHelper

dbHelper = PersonDatabaseHelper()

dbHelper.deleteTable()
dbHelper.createTable()

dbHelper.insertPersons(persons)

dbHelper.commit()

inserted 392 rows into table person


In [14]:
for person in dbHelper.selectAll():
    print(person)

(1, 'Fares Abawi', '', 'Universität Hamburg', 'WTM', None)
(2, 'Rana Abdullah', '', 'Universität Hamburg', 'SEMS', None)
(3, 'Daniel Ahlers', '', 'Universität Hamburg', 'TAMS', None)
(4, 'Kyra Ahrens', '', 'Universität Hamburg', 'WTM', None)
(5, 'Hassan Ali', '', 'Universität Hamburg', 'WTM', None)
(6, 'Philipp Allgeuer', 'Dr.', 'Universität Hamburg', 'WTM', None)
(7, 'Selenia Anastasi', '', 'Universität Hamburg', 'LT', None)
(8, 'Jakob Andersen', '', 'Universität Hamburg', 'MAST', None)
(9, 'Johanna Ansohn Mcdougall', '', 'Universität Hamburg', 'SVS', None)
(10, 'Saba Anwar', '', 'Universität Hamburg', 'LT', None)
(11, 'Oscar Javier Ariza Nunez', 'Dr.', 'Universität Hamburg', 'HCI', None)
(12, 'Anne Awizen', '', 'Universität Hamburg', 'DBIS', None)
(13, 'Abinew Ali Ayele', '', 'Universität Hamburg', 'LT', None)
(14, 'Christian Bähnisch', '', 'Universität Hamburg', 'HITeC', None)
(15, 'Debayan Banerjee', '', 'Universität Hamburg', 'LT', None)
(16, 'Jan Baumbach', 'Prof. Dr.', 'Universi